Import des librairies
- pixiedust (Kernel Python/Scala)
- time

In [1]:
#import pixiedust
import time
import os
import tweepy
import json



Configuration du nom du fichier utilisé

In [15]:
%env TWEET_JSON_FILENAME=tweet_{time.time()}.json

env: TWEET_JSON_FILENAME=tweet_1520508713.3911748.json


In [18]:
#Variables that contains the user credentials to access Twitter API
access_token = "3938549961-5PLnzjbxwjjMgIKeiG8Wj0LW9yPu5p60mNTZG3c"
access_token_secret = "dcZcTh1FQegfELoQlagngT9JUm6vu6XxIbJiYboRiOsHX"
consumer_key = "n9zZhnXVOnU6W3sFIIQXErCo7"
consumer_secret = "kAE0Uz68Kex2MAkPgo1xB9DzdDaf9d5U2zW3EubZ5fFsFjt98j"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

keyword = "#Trump"
tweets = tweepy.Cursor(api.search, q=keyword).items(10)
json_tweets = [json.dumps(tweet._json) for tweet in tweets]
json_str = "\n\r".join(json_tweets)

filename = "tweets-database/"+os.environ['TWEET_JSON_FILENAME']
with open(filename, "w") as f:
    f.write(json_str)

In [19]:
%%bash
cd Algo1
cp trump.json ../tweets-database/$TWEET_JSON_FILENAME

Mise à jour de l'algorithme sur le Datacenter

In [4]:
%%bash
cd Algo1
sbt "push"

[info] Loading settings from idea.sbt ...
[info] Loading global plugins from /Users/Frego/.sbt/1.0/plugins
[info] Loading settings from plugins.sbt ...
[info] Loading project definition from /Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/project
[info] Loading settings from build.sbt ...
[info] Set current project to SparkSenti (in build file:/Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/)
[info] Exclude scala-library-2.11.8.jar from the package
[info] Compiling 1 Scala source to /Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/target/scala-2.11/classes ...
[warn] Class org.joda.convert.FromString not found - continuing with a stub.
[warn] Class org.joda.convert.FromString not found - continuing with a stub.
[warn] Class org.joda.convert.ToString not found - continuing with a stub.
[warn] Class org.joda.convert.FromString not found - continuing with a stub.
[warn] Class org.joda.convert.ToString not found - continuing wit

Envoie des tweets collectés sur le datacenter

In [21]:
%%bash
cp tweets-database/$TWEET_JSON_FILENAME Algo1/$TWEET_JSON_FILENAME
cd Algo1
sbt "put $TWEET_JSON_FILENAME"

[info] Loading settings from idea.sbt ...
[info] Loading global plugins from /Users/Frego/.sbt/1.0/plugins
[info] Loading settings from plugins.sbt ...
[info] Loading project definition from /Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/project
[info] Loading settings from build.sbt ...
[info] Set current project to SparkSenti (in build file:/Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/)
spawn scp tweet_1520508713.3911748.json fregosi1@156.18.90.100:.
fregosi1@156.18.90.100's password: 
tweet_1520508713.3911748.json                 100%   45KB 663.7KB/s   00:00    
spawn ssh fregosi1@156.18.90.100
fregosi1@156.18.90.100's password: 

Welcome to Bright release         7.3

                                                        Based on CentOS Linux 7
                                                                    ID: #000002

Use the following commands to adjust your environment:

'module avail'            - show available modules
'mod

Execution de l'analyse de sentiments des tweets colléctés sur le datacenter

In [22]:
%%bash
cd Algo1
rm -rf $TWEET_JSON_FILENAME
sbt "submit $TWEET_JSON_FILENAME"

[info] Loading settings from idea.sbt ...
[info] Loading global plugins from /Users/Frego/.sbt/1.0/plugins
[info] Loading settings from plugins.sbt ...
[info] Loading project definition from /Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/project
[info] Loading settings from build.sbt ...
[info] Set current project to SparkSenti (in build file:/Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/)
spawn ssh fregosi1@156.18.90.100
fregosi1@156.18.90.100's password: 

Welcome to Bright release         7.3

                                                        Based on CentOS Linux 7
                                                                    ID: #000002

Use the following commands to adjust your environment:

'module avail'            - show available modules
'module add <module>'     - adds a module to your environment for this session
'module initadd <module>' - configure module to be loaded at every login

--------------------------------

Récupération du resultat de l'analyse : fichier text (id_tweet sentiment)

Mapping des données pour Viz 

Visualisation